# Box Bot Project Roadmap

This notebook recreates the `box_bot_description` project following the specific development roadmap.

### Roadmap
**Phase 1: Setup and Robot Modeling**
* Step 1: Create ROS 2 Workspace and Package
* Step 2: Define the Robot Model (URDF)
* Step 3: Configure Gazebo and Launch Files

**Phase 2: Simulation and Basic Control**
* Step 4: Build and Verify Simulation
* Step 5: Test Basic Movement (Topics)

**Phase 3: Implementing Communication Mechanisms**
* Step 6: Implement Core Nodes (Python) - Obstacle Avoider
* Step 7: Final Launch Integration

**Phase 4: Navigation and Inspection Logic**
* Step 8: Implement Navigation (Nav2)
* Step 9: Program Autonomous Mission

## 🚀 Phase 1: Setup and Robot Modeling

### Step 1: Create ROS 2 Workspace and Package

In [ ]:
!mkdir -p src
!ros2 pkg create --build-type ament_python box_bot_description --dependencies rclpy sensor_msgs geometry_msgs launch_ros nav2_msgs --destination-directory src

### Step 2: Define the Robot Model (URDF)
We create the `urdf` directory and the `box_bot.urdf` file describing the physical robot.

In [ ]:
!mkdir -p src/box_bot_description/urdf

In [ ]:
%%writefile src/box_bot_description/urdf/box_bot.urdf
<?xml version="1.0" ?>
<robot name="box_bot">

  <!-- Base Link -->
  <link name="base_link">
    <visual>
      <geometry>
        <box size="0.5 0.5 0.2"/>
      </geometry>
      <material name="blue">
        <color rgba="0 0 1 1"/>
      </material>
    </visual>
    <collision>
      <geometry>
        <box size="0.5 0.5 0.2"/>
      </geometry>
    </collision>
    <inertial>
      <mass value="5.0"/>
      <inertia ixx="0.1" ixy="0.0" ixz="0.0" iyy="0.1" iyz="0.0" izz="0.1"/>
    </inertial>
  </link>

  <!-- Base Footprint (Required for Nav2) -->
  <link name="base_footprint"/>
  <joint name="base_joint" type="fixed">
    <parent link="base_link"/>
    <child link="base_footprint"/>
    <origin xyz="0 0 -0.1" rpy="0 0 0"/>
  </joint>

  <!-- Lidar Link -->
  <link name="laser_frame">
    <visual>
      <geometry>
        <cylinder radius="0.05" length="0.05"/>
      </geometry>
      <material name="red">
        <color rgba="1 0 0 1"/>
      </material>
    </visual>
  </link>

  <joint name="laser_joint" type="fixed">
    <parent link="base_link"/>
    <child link="laser_frame"/>
    <origin xyz="0 0 0.125" rpy="0 0 0"/>
  </joint>

</robot>

### Step 3: Configure Gazebo and Launch Files
We need to update `setup.py` to handle our resource files and create the simulation launch file.

In [ ]:
%%writefile src/box_bot_description/setup.py
from setuptools import find_packages, setup
import os
from glob import glob

package_name = 'box_bot_description'

setup(
    name=package_name,
    version='0.0.0',
    packages=find_packages(exclude=['test']),
    data_files=[
        ('share/ament_index/resource_index/packages',
            ['resource/' + package_name]),
        ('share/' + package_name, ['package.xml']),
        # Include all launch files
        (os.path.join('share', package_name, 'launch'), glob(os.path.join('launch', '*launch.[pxy][yma]*'))),
        # Include all URDF files
        (os.path.join('share', package_name, 'urdf'), glob(os.path.join('urdf', '*.urdf'))),
        # Include world files
        (os.path.join('share', package_name, 'world'), glob(os.path.join('world', '*.world'))),
        # Include maps
        (os.path.join('share', package_name, 'maps'), glob('maps/*')),
        # Include config
        (os.path.join('share', package_name, 'config'), glob(os.path.join('config', '*.yaml'))),
    ],
    install_requires=['setuptools'],
    zip_safe=True,
    maintainer='your_name',
    maintainer_email='your_email@todo.todo',
    description='ROS 2 package for a simple box robot simulation',
    license='Apache License 2.0',
    tests_require=['pytest'],
    entry_points={
        'console_scripts': [
            'obstacle_avoider = box_bot_description.obstacleavoider:main',
        ],
    },
)

In [ ]:
!mkdir -p src/box_bot_description/launch
!mkdir -p src/box_bot_description/world

In [ ]:
%%writefile src/box_bot_description/world/my_world.world
<?xml version="1.0" ?>
<sdf version="1.6">
  <world name="default">
    <include>
      <uri>https://fuel.gazebosim.org/1.0/OpenRobotics/models/Sun</uri>
    </include>
    <include>
      <uri>https://fuel.gazebosim.org/1.0/OpenRobotics/models/Ground Plane</uri>
    </include>
  </world>
</sdf>

In [ ]:
%%writefile src/box_bot_description/launch/sim.launch.py
import os
from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch.actions import IncludeLaunchDescription
from launch.launch_description_sources import PythonLaunchDescriptionSource
from launch_ros.actions import Node

def generate_launch_description():
    package_name = 'box_bot_description'
    pkg_share = get_package_share_directory(package_name)
    urdf_path = os.path.join(pkg_share, 'urdf', 'box_bot.urdf')
    world_path = os.path.join(pkg_share, 'world', 'my_world.world')

    with open(urdf_path, 'r') as infp:
        robot_desc = infp.read()

    # Launch Gazebo
    gazebo = IncludeLaunchDescription(
        PythonLaunchDescriptionSource([os.path.join(
            get_package_share_directory('ros_gz_sim'), 'launch', 'gz_sim.launch.py')]),
        launch_arguments={'gz_args': f'-r {world_path}'}.items(),
    )

    # Robot State Publisher
    robot_state_publisher = Node(
        package='robot_state_publisher',
        executable='robot_state_publisher',
        output='screen',
        parameters=[{'robot_description': robot_desc, 'use_sim_time': True}]
    )

    # Spawn Entity
    spawn_entity = Node(
        package='ros_gz_sim',
        executable='create',
        arguments=['-topic', 'robot_description', '-name', 'box_bot', '-z', '0.1'],
        output='screen'
    )

    # Bridge
    bridge = Node(
        package='ros_gz_bridge',
        executable='parameter_bridge',
        arguments=[
            '/scan@sensor_msgs/msg/LaserScan[gz.msgs.LaserScan',
            '/cmd_vel@geometry_msgs/msg/Twist]gz.msgs.Twist',
            '/model/box_bot/tf@tf2_msgs/msg/TFMessage[gz.msgs.Pose_V'
        ],
        remappings=[('/model/box_bot/tf', '/tf')],
        parameters=[{'use_sim_time': True}],
        output='screen'
    )

    return LaunchDescription([gazebo, robot_state_publisher, spawn_entity, bridge])

## 🚀 Phase 2: Simulation and Basic Control

### Step 4: Build and Verify Simulation

In [ ]:
!colcon build --symlink-install

### Step 5: Test Basic Movement
To test, you would open a terminal and run:
`ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}, angular: {z: 0.0}}"`

## 🧠 Phase 3: Implementing Communication Mechanisms

### Step 6: Implement Core Nodes (Python) - Obstacle Avoider

In [ ]:
%%writefile src/box_bot_description/box_bot_description/obstacleavoider.py
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist
from rclpy.qos import QoSProfile, ReliabilityPolicy

class ObstacleAvoider(Node):
    def __init__(self):
        super().__init__('obstacle_avoider')
        self.publisher_ = self.create_publisher(Twist, 'cmd_vel', 10)
        qos_profile = QoSProfile(depth=10, reliability=ReliabilityPolicy.BEST_EFFORT)
        self.subscription = self.create_subscription(
            LaserScan, '/scan', self.listener_callback, qos_profile)
        self.get_logger().info('Obstacle Avoider Node Started')

    def listener_callback(self, msg):
        cmd = Twist()
        # Simple logic: if center ray is close, turn; else move forward
        mid_index = len(msg.ranges) // 2
        if msg.ranges[mid_index] < 1.0:
            cmd.angular.z = 0.5
        else:
            cmd.linear.x = 0.5
        self.publisher_.publish(cmd)

def main(args=None):
    rclpy.init(args=args)
    node = ObstacleAvoider()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()

if __name__ == '__main__':
    main()

## 🎯 Phase 4: Navigation and Inspection Logic

### Step 8: Implement Navigation (Nav2)
We create the `navigation.launch.py` and necessary config files.

In [ ]:
!mkdir -p src/box_bot_description/config
!mkdir -p src/box_bot_description/maps

In [ ]:
%%writefile src/box_bot_description/config/nav2_params.yaml
amcl:
  ros__parameters:
    use_sim_time: True
    alpha1: 0.2
    alpha2: 0.2
    alpha3: 0.2
    alpha4: 0.2
    alpha5: 0.2

bt_navigator:
  ros__parameters:
    use_sim_time: True
    global_frame: map
    robot_base_frame: base_link
    odom_topic: /odom
    bt_loop_duration: 10
    default_server_timeout: 20

controller_server:
  ros__parameters:
    use_sim_time: True

planner_server:
  ros__parameters:
    use_sim_time: True
    expected_planner_frequency: 20.0
    planner_plugins: ["GridBased"]
    GridBased:
      plugin: "nav2_navfn_planner/NavfnPlanner"
      tolerance: 0.5
      use_astar: false
      allow_unknown: true

In [ ]:
%%writefile src/box_bot_description/launch/navigation.launch.py
import os
from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    package_name = 'box_bot_description'
    pkg_share = get_package_share_directory(package_name)
    map_file = os.path.join(pkg_share, 'maps', 'my_new_mapp.yaml')
    nav2_params_path = os.path.join(pkg_share, 'config', 'nav2_params.yaml')

    return LaunchDescription([
        Node(package='nav2_map_server', executable='map_server', name='map_server',
             parameters=[nav2_params_path, {'yaml_filename': map_file}]),
        Node(package='nav2_amcl', executable='amcl', name='amcl',
             parameters=[nav2_params_path]),
        Node(package='nav2_planner', executable='planner_server', name='planner_server',
             parameters=[nav2_params_path]),
        Node(package='nav2_controller', executable='controller_server', name='controller_server',
             parameters=[nav2_params_path]),
        Node(package='nav2_behaviors', executable='behavior_server', name='behavior_server',
             parameters=[nav2_params_path]),
        Node(package='nav2_bt_navigator', executable='bt_navigator', name='bt_navigator',
             parameters=[nav2_params_path]),
        Node(package='nav2_lifecycle_manager', executable='lifecycle_manager', name='lifecycle_manager_navigation',
             parameters=[{'autostart': True}, {'node_names': ['map_server', 'amcl', 'planner_server', 'controller_server', 'behavior_server', 'bt_navigator']}])
    ])